In [1]:
import torch
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.nn.utils.rnn as rnn_utils
import json
import sys
import random
import numpy as np
import pickle

import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

from Vocab import Vocab
from Tokenize import Tokenize


fo='./models/'

In [5]:
class classifier(nn.Module):
    def __init__(self, use_cuda, hidden_dim, input_dim, vocab_size):
        super(classifier, self).__init__()
        self.use_cuda = use_cuda
        self.input_dim=input_dim
        self.hidden_dim=hidden_dim
        self.lstm=torch.nn.LSTM(input_size=self.input_dim, 
                                hidden_size= self.hidden_dim, 
                                bidirectional=True,
                                batch_first=True
                               )
        self.softmax=nn.Softmax()
        self.fc=nn.Linear(hidden_dim, vocab_size)
        self.fc1=nn.Linear(hidden_dim, 1000)
        self.fc2=nn.Linear(1000, vocab_size)
        self.activate_func=nn.ReLU()
        #embedding
        self.embed=nn.Embedding(vocab_size, input_dim)
        #loss
        self.cost_func = nn.CrossEntropyLoss()
        
        self.label = 0
        self.label_embedding = 0
        
    def order(self, inputs, inputs_len):
        
        inputs_len, sort_ids = torch.sort(inputs_len, descending=True)
        #print(inputs.shape, sort_ids.shape, inputs_len.shape)
        
        if self.use_cuda:
            inputs = inputs.index_select(0, Variable(sort_ids).cuda())
        else:
            inputs = inputs.index_select(0, Variable(sort_ids))
        
        _, true_order_ids = torch.sort(sort_ids, descending=False)
        
        return inputs, inputs_len, true_order_ids
    #
    def forward(self, inputs, inputs_len, is_fix_embedding, embedding_update_num):
        inputs = Variable(inputs)
        if self.use_cuda:
            inputs=inputs.cuda()
            
        inputs, sort_len, true_order_ids = self.order(inputs, inputs_len)
        
        if is_fix_embedding:
            for para in self.embed.parameters():
                para.requires_grad=False
                #print('ye')
            cnt=0
            while(cnt<=embedding_update_num):
                cnt+=1
                rand_range, _ = self.label_embedding.shape
                randn = random.randint(0,rand_range-1)
                token_idx = self.label[randn]
                token_embedding = self.label_embedding[randn]
                self.embed.weight.data[token_idx.data]=token_embedding
            print(cnt)
        else:
            for para in self.embed.parameters():
                para.requires_grad=True
                
#         print(self.embed.weight.data[0])
#         self.embed.weight.data[0]=torch.LongTensor(([2]*400))
#         print(self.embed.weight.data[0])
#         print(self.embed.weight)
            
        in_vec=self.embed(inputs)
        packed = rnn_utils.pack_padded_sequence(input=in_vec, lengths=list(sort_len), batch_first =True)
        
        outputs, (hn,cn) = self.lstm(packed)
        print(hn.shape, cn.shape)
        
        hn = torch.squeeze(hn[-1,:,:])
        
        if self.use_cuda:
            hn = hn.index_select(0, Variable(true_order_ids).cuda())
        else:
            hn = hn.index_select(0, Variable(true_order_ids))
#         print(in_vec.size())
#         print(outputs.size())
#         print(hn.size())
        self.label_embedding=hn
        return self.softmax(self.fc(hn))

    def get_loss(self, predicts, labels):
        labels=Variable(labels).long()
        
        if self.use_cuda:
            labels = labels.cuda()
        self.label=labels
        return self.cost_func(predicts, labels)
        
print('ye')        

ye


In [6]:
# fi_path='./vocab.mean.t2s.txt'
# vocab=Vocab(fi_path)
# vocab.build_vocab()
# vocab.print_vocab()
# vocab.line_length_statistics(filter_rate=0.8)
# tokenize=Tokenize(fi_path, vocab, length_cut=25)
# explains_token, labels_token=tokenize.word_tokenize()

with open('./tokenize.txt','rb') as f:
    tokenize=pickle.load(f)
f.close()
vocab=tokenize.vocab
#hyper par
use_cuda=torch.cuda.is_available()
hidden_dim=50
input_dim=50
vocab_size=len(vocab.word2idx)
lr=0.003

model=classifier(
                use_cuda=use_cuda, 
                hidden_dim=hidden_dim, input_dim=input_dim, 
                vocab_size=vocab_size
                )
#load model trained and continue to train the parameters
# model_trained = torch.load('./models/lr0.008-in_dim100-loss8.415006-train_acc 0.29.pkl')
# model.load_state_dict(model_trained)

if use_cuda:
    model.cuda()

inputs=tokenize.explains_token
labels=tokenize.labels_token
inputs_len=tokenize.sentence_lengths

inputs_=torch.LongTensor(inputs)
labels_=torch.LongTensor(labels)
inputs_len_=torch.LongTensor(inputs_len)

print(labels_.shape, inputs_.shape, inputs_len_.shape)

torch.Size([17388]) torch.Size([17388, 17]) torch.Size([17388])


In [7]:
optimizer = optim.Adam(model.parameters(), lr=lr)
iterate=0
iter_n=1500
while(iterate<iter_n):
    if iterate%2==0:
        inputs_=torch.LongTensor(inputs[0:int(len(inputs)/2)])
        labels_=torch.LongTensor(labels[0:int(len(inputs)/2)])
        inputs_len_=torch.LongTensor(inputs_len[0:int(len(inputs)/2)])
    else:
        inputs_=torch.LongTensor(inputs[int(len(inputs)/2):-1])
        labels_=torch.LongTensor(labels[int(len(inputs)/2):-1])
        inputs_len_=torch.LongTensor(inputs_len[int(len(inputs)/2):-1])

#     inputs_=torch.LongTensor(inputs[0:1000])
#     labels_=torch.LongTensor(labels[0:1000])
#     inputs_len_=torch.LongTensor(inputs_len[0:1000])
    
    iterate+=1
    if iterate<=10:
        predicts=model(inputs_, inputs_len_,0,1000)
    else:
        predicts=model(inputs_, inputs_len_,1,2000)
    optimizer.zero_grad()
    loss=model.get_loss(predicts, labels_)
    #optimize
    loss.backward()
    optimizer.step()
    if iterate%1==0:
        if use_cuda:
            preds=predicts.data.cpu()
        else:
            preds=predicts.data
        (pred, idxs) = torch.max(preds, 1)
        
        print('iterate %s times, '%iterate, 
              'loss: %5.6f' %loss.data[0], 
              'train_right: %s' %sum(labels_==idxs),
              'train_acc: %5.6f' %(sum(labels_==idxs)/len(preds))
             )
        
    #
    if iterate%100==0:
        torch.save(model.state_dict(), 
                   fo+'lr{}-in_dim{}-loss{:5.6f}-train_acc{:5.2f}.pkl'.format(lr, input_dim, loss.data[0],
                                                                             sum(labels_==idxs)/len(preds))
                  )

torch.Size([2, 8694, 50]) torch.Size([2, 8694, 50])


/data1/hmx/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:84: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


iterate 1 times,  loss: 8.702859 train_right: 0 train_acc: 0.000000
torch.Size([2, 8693, 50]) torch.Size([2, 8693, 50])
iterate 2 times,  loss: 8.702858 train_right: 3 train_acc: 0.000345
torch.Size([2, 8694, 50]) torch.Size([2, 8694, 50])
iterate 3 times,  loss: 8.702858 train_right: 0 train_acc: 0.000000
torch.Size([2, 8693, 50]) torch.Size([2, 8693, 50])
iterate 4 times,  loss: 8.702857 train_right: 2 train_acc: 0.000230
torch.Size([2, 8694, 50]) torch.Size([2, 8694, 50])
iterate 5 times,  loss: 8.702858 train_right: 0 train_acc: 0.000000
torch.Size([2, 8693, 50]) torch.Size([2, 8693, 50])
iterate 6 times,  loss: 8.702857 train_right: 2 train_acc: 0.000230
torch.Size([2, 8694, 50]) torch.Size([2, 8694, 50])
iterate 7 times,  loss: 8.702857 train_right: 0 train_acc: 0.000000
torch.Size([2, 8693, 50]) torch.Size([2, 8693, 50])
iterate 8 times,  loss: 8.702857 train_right: 2 train_acc: 0.000230
torch.Size([2, 8694, 50]) torch.Size([2, 8694, 50])
iterate 9 times,  loss: 8.702857 train_r

SystemError: <built-in function isinstance> returned a result with an error set